# Loading Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Reading the Data

In [2]:
heart = pd.read_csv('framingham.csv')
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


# Defining the X & Y

In [3]:
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

skf = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 1, random_state = 42)

In [4]:
X.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,4238.000000,4238.000000,4188.000000,4219.000000,4237.000000
mean,49.584946,0.494101,236.721585,25.802008,75.878924
std,8.572160,0.500024,44.590334,4.080111,12.026596
min,32.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.070000,68.000000
50%,49.000000,0.000000,234.000000,25.400000,75.000000
75%,56.000000,1.000000,263.000000,28.040000,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


# Random Forest

In [5]:
md1 = Pipeline([('imputer', KNNImputer(n_neighbors = 5, weights = 'distance', 
                                       add_indicator = True)), 
                ('RF', RandomForestClassifier(n_estimators = 100, 
                                              max_depth = 3))])
RF_cv = cross_val_score(md1, X, Y, cv = skf, scoring = 'roc_auc', n_jobs = -1)

print(f"The average 10-folds cross-validation ROC-AUC score of the RF model is {RF_cv.mean()}")

The average 10-folds cross-validation ROC-AUC score of the RF model is 0.6822786002636716


# Gradient Boosting

In [6]:
md2 = Pipeline([('imputer', KNNImputer(n_neighbors = 5, weights = 'distance', 
                                       add_indicator = True)), 
                ('GB', GradientBoostingClassifier(n_estimators = 100, 
                                                  max_depth = 3,
                                                  learning_rate = 0.1))])
GB_cv = cross_val_score(md2, X, Y, cv = skf, scoring = 'roc_auc', n_jobs = -1)

print(f"The average 10-folds cross-validation ROC-AUC score of the GB model is {GB_cv.mean()}")

The average 10-folds cross-validation ROC-AUC score of the GB model is 0.6738533952373937


In [7]:
# Based on my results, RF is the best model out of the two considered.